# Alle Karteringen omzetten

Maar voor nu nog even enkel groningen

### Imports etc

In [1]:
from pathlib import Path
import pandas as pd
import geopandas as gpd

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_was_wordt_lijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
from veg2hab.fgr import FGR

pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 8)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Omzetclasses instantieren

In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_was_wordt_lijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)

path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.csv")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)

fgr = FGR(Path("../data/bronbestanden/FGR.json"))

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide


### Inladen benodigde informatie

**Overzicht_vegetatiekarteringen.xlsx** is uit de drive en weet van karteringen de provincie, naam, shapefile pad en of het een access of een shapefile kartering is.

**accesskarteringinfo.xlsx** bevat voor de access karteringen de informatie die nodig is om een access kartering in te laden.

**shapefilekarteringinfo.xlsx** bevat voor de shapefile karteringen de informatie die nodig is om een shapefile kartering in te laden.


In [3]:
# info over de karteringen
overzicht = pd.read_excel(
    "../data/Overzicht_vegetatiekarteringen.xlsx", 
    sheet_name="Vegetatiekarteringen", 
    engine='openpyxl',
    usecols=[
        "provincie", 
        "naam_kartering", 
        "path_shapes", 
        "landelijke_typologie beschikbaar?", 
        "locatie_landelijke_typologie",
    ],
    dtype="string"
)

# benodigde info voor de access karteringen
access_info = pd.read_excel(
    "../data/accesskarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "path_csvs",
        "elm_id_name",
    ],
    dtype="string"
)

# benodigde info voor de shapefile karteringen
shape_info = pd.read_excel(
    "../data/shapefilekarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "ElmID_col",
        "VvN_col",
        "SBB_col",
        "vegtype_split_char",
        "datum_col",
        "opmerking_col",
    ],
    dtype="string"
)
# Convert all pd.NA to None to prevent having to do "if not pd.isna()" everywhere
shape_info = shape_info.applymap(lambda x: None if pd.isna(x) else x)

overzicht = overzicht[overzicht["landelijke_typologie beschikbaar?"] == "Ja"]
overzicht.path_shapes = overzicht.path_shapes.apply(lambda x: "../testing/vegetatiekarteringen" + x[1:])

access_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "access"]
access_overzicht = access_overzicht.join(access_info.set_index("naam_kartering"), on="naam_kartering")
access_overzicht.path_csvs = access_overzicht.path_csvs.apply(lambda x: "../testing/vegetatiekarteringen" + x[1:])

shapefile_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "in shapes"]
shapefile_overzicht = shapefile_overzicht.join(shape_info.set_index("naam_kartering"), on="naam_kartering")

### Beddekking/Percentages in shapefilekarteringen...

...doen we voor nu nog even niets mee.

Ze zijn of altijd 100 of ook boven de 100, en ze zijn toch niet per vegtypeinfo beschikbaar maar per vlak

In [4]:
# for row in shapefile_overzicht.itertuples():
#     t = gpd.read_file(row.path_shapes)
#     print(row.naam_kartering)
#     print(t.BEDEKKING.value_counts())

In [4]:
shapefile_overzicht

,provincie,naam_kartering,path_shapes,landelijke_typologie beschikbaar?,...,SBB_col,vegtype_split_char,datum_col,opmerking_col
0,Drenthe,Asserbos2015,../testing/vegetatiekarteringen/DR/Asserbos201...,Ja,...,None,None,None,None
1,Drenthe,BG_Drenthe2014,../testing/vegetatiekarteringen/DR/BG_Drenthe2...,Ja,...,None,None,None,None
2,Drenthe,BG_Holtingerveld_bossen2016,../testing/vegetatiekarteringen/DR/BG_Holtinge...,Ja,...,None,None,None,None
3,Drenthe,BG_Hondsrug2019,../testing/vegetatiekarteringen/DR/BG_Hondsrug...,Ja,...,None,None,None,None
4,Drenthe,BG_meerdere_terreinen2020,../testing/vegetatiekarteringen/DR/BG_meerdere...,Ja,...,None,None,None,None
5,Drenthe,BG_Nolde2018,../testing/vegetatiekarteringen/DR/BG_Nolde201...,Ja,...,None,None,None,None
6,Drenthe,DEF_Holtingerveld2013,../testing/vegetatiekarteringen/DR/DEF_Holting...,Ja,...,None,None,None,None
7,Drenthe,GL_Zuidlaardermeer2019,../testing/vegetatiekarteringen/DR/GL_Zuidlaar...,Ja,...,None,None,None,None
8,Drenthe,HDL_De_Stroeten2016,../testing/vegetatiekarteringen/DR/HDL_De_Stro...,Ja,...,None,None,None,None
9,Drenthe,HDL_Hijkerveld-Smilder_oosterveld2018,../testing/vegetatiekarteringen/DR/HDL_Hijkerv...,Ja,...,None,None,None,None


In [6]:
for row in shapefile_overzicht.itertuples():
    gdf = gpd.read_file(row.path_shapes)
    print(f"------- {row.naam_kartering} -------")
    print(gdf.head)

------- Asserbos2015 -------
<bound method NDFrame.head of     OBJECTID_1  OBJECTID            IDENTIFICA   TOP10_ID        TYPELANDGE  \
0            0    239949  NL.TOP10NL.107100242  107100242          grasland   
1            0    240902  NL.TOP10NL.107104589  107104589     bos: naaldbos   
2            0    249043  NL.TOP10NL.107105434  107105434  bos: gemengd bos   
3            0    248409  NL.TOP10NL.107101137  107101137  bos: gemengd bos   
4            0    250432  NL.TOP10NL.107101263  107101263  bos: gemengd bos   
..         ...       ...                   ...        ...               ...   
69           0    604373  NL.TOP10NL.127854007  127854007      bos: loofbos   
70           0    604456  NL.TOP10NL.127854012  127854012          grasland   
71           0    604089  NL.TOP10NL.127853975  127853975     bos: naaldbos   
72           0    604090  NL.TOP10NL.127853976  127853976      bos: loofbos   
73           0    604091  NL.TOP10NL.127853977  127853977     bos: naald

KeyboardInterrupt: 

### Karteringen inladen

In [ ]:
karteringen = {}
paths = {}
for row in access_overzicht.itertuples():
    print(row.naam_kartering)
    try:
        karteringen[row.naam_kartering] = Kartering.from_access_db(Path(row.path_shapes), row.elm_id_name, Path(row.path_csvs))
        
    except ValueError as e:
        print(e)
        print(f"!!! {row.naam_kartering} wordt overgeslagen !!!")

# for row in shapefile_overzicht.itertuples():
#     karteringen[row.naam_kartering] = Kartering.from_shapefile(
#         Path(row.path_shapes),
#         row.ElmID_col,
#         VvN_col=row.VvN_col,
#         SBB_col=row.SBB_col, 
#         vegtype_split_char=row.vegtype_split_char, 
#         datum_col=row.datum_col, 
#         opmerking_col=row.opmerking_col
#     )

NM_Fochteloerveen_2020
NM_DCR_2013
Er zijn 476 vlakken zonder VegTypeInfo in ../testing/vegetatiekarteringen/DR Extra/NM_DCR2013/NM_Fochteloerveen_DCR2013.shp. Deze worden verwijderd.
De eerste paar ElmID van de verwijderde vlakken zijn: [nan, nan, nan, nan, nan]
NM_Westerbork2017


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:416: UserWarning: CRS van ../testing/vegetatiekarteringen/DR/NM_Westerbork2017/NM_Westerbork2017.shp was EPSG:None en is nu omgezet naar EPSG:28992
  f"CRS van {shape_path} was EPSG:{gdf.crs.to_epsg()} en is nu omgezet naar EPSG:28992"


SBB0817_Grolloerveen2013
SBB0863_Elperstroom_Schoonloo2012
SBB0936_DrentseAa2015_16


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:411: UserWarning: CRS van ../testing/vegetatiekarteringen/DR/SBB0936_DrentseAa2015_16/SBB0936_DrentseAa2015_16.shp was None en is nu gelezen als EPSG:28992
  f"CRS van {shape_path} was None en is nu gelezen als EPSG:28992"


Fr 0752_Bakkefean_2009


...

Slotplaats_2016
Ameland_2010


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:411: UserWarning: CRS van ../testing/vegetatiekarteringen/FR/Duinen_Ameland/Ameland_2010/vlakken.shp was None en is nu gelezen als EPSG:28992
  f"CRS van {shape_path} was None en is nu gelezen als EPSG:28992"


Schiermonnikoog_2015
Schiermonnikoog_2017
Terschelling_2012


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:411: UserWarning: CRS van ../testing/vegetatiekarteringen/FR/Duinen_Terschelling/Terschelling_2012/vlakken.shp was None en is nu gelezen als EPSG:28992
  f"CRS van {shape_path} was None en is nu gelezen als EPSG:28992"


Vlieland_2013
Lauwersmeer_Fryslan_2015
Rottige_Meenthe_Brandemeer_2013
Sneekermeergebied_2009_Terkaplester puollen
Bakkefean_2022
Snitsermar_2021
Deelen_2022
Tjonger_2021
Er zijn 179 vlakken zonder VegTypeInfo in ../testing/vegetatiekarteringen/FR Extra/Tjonger_2021_SBB/Shapefiles/Vlakken/Vlakken_inclSBBtyp.shp. Deze worden verwijderd.
De eerste paar ElmID van de verwijderde vlakken zijn: [1, 2, 3, 4, 5]
Witte en zwarte brekken_2021
Er zijn 16 vlakken zonder VegTypeInfo in ../testing/vegetatiekarteringen/FR Extra/Witte en zwarte brekken Veg&Flora 2021/Shapefiles/Shapefiles/Vlakken.shp. Deze worden verwijderd.
De eerste paar ElmID van de verwijderde vlakken zijn: [6, 19, 20, 33, 34]
NM vegetatiekartering RuitenAa2020
Er is geen 1 op 1 relatie tussen Elmid in de shapefile en ElmID in de Element.csv. Er zitten dubbelingen in de shapefile, bijvoorbeeld Elmid: [8174, 11023, 11051].
!!! NM vegetatiekartering RuitenAa2020 wordt overgeslagen !!!
SBB Duurswold2013
SBB Lauwersmeer 2015
SBB Weste

### Karteringen omzetten

In [ ]:
for kartering in karteringen.values():
    kartering.apply_wwl(wwl)

In [ ]:
for kartering in karteringen.values():
    kartering.apply_deftabel(dt)

In [ ]:
for kartering in karteringen.values():
    kartering.check_mitsen(fgr)

### Karteringen wegschrijven

In [ ]:
for gebied in (karteringen.keys()):
    print(gebied)
    p = Path(f"../testing/omgezette_vegetatiekarteringen/GR/{gebied}.shp")
    karteringen[gebied].final_format_to_file(p)

NM_Fochteloerveen_2020
NM_DCR_2013
NM_Westerbork2017
SBB0817_Grolloerveen2013
SBB0863_Elperstroom_Schoonloo2012
SBB0936_DrentseAa2015_16


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:711: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final.to_file(path)


Fr 0752_Bakkefean_2009
Slotplaats_2016
Ameland_2010
Schiermonnikoog_2015
Schiermonnikoog_2017
Terschelling_2012
Vlieland_2013
Lauwersmeer_Fryslan_2015
Rottige_Meenthe_Brandemeer_2013
Sneekermeergebied_2009_Terkaplester puollen
Bakkefean_2022
Snitsermar_2021
Deelen_2022
Tjonger_2021
Witte en zwarte brekken_2021
SBB Duurswold2013
SBB Lauwersmeer 2015
SBB Westerwolde 2020
SBB ZWK 2010 0815_Trimunt_2010
SGL Zuidlaardermeer 2019
